In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from fsl_mrs.utils.plotting import FID2Spec
import fsl_mrs.utils.mrs_io as mrs_io
from fsl_mrs.utils.preproc import nifti_mrs_proc as proc
from fsl_mrs.utils.plotting import plotly_fit
from fsl_mrs.utils import misc
import fsl_mrs.core as core


part = "004"
from pathlib import Path

# Path to the directory 
NOTEBOOK_DIR = Path().resolve()
print(NOTEBOOK_DIR)

#Load Data

off_sup = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_fitting/mPress/{part}/off_sup.nii.gz')
off_unsup = mrs_io.read_FID(rf'{NOTEBOOK_DIR}/Data_for_fitting/mPress/{part}/off_unsup.nii.gz')

In [ ]:
#Phase correct based on the water

off_unsup = proc.shift_to_reference(off_unsup, 4.65, (4.5,4.8))
off_unsup = proc.phase_correct(off_unsup, (4.55, 4.75))


plt.figure()
_ = off_sup.plot(ppmlim= (0,4.2))
_ = off_unsup.plot(ppmlim=(0,4.2))

In [ ]:
#Fit Water suppressed reference data

basis_off = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/uzay_svs_mpress_68_off_h2o')
basis_off.remove_fid_from_basis('H2O')

supmrs_off = off_sup.mrs(basis=basis_off)
supmrs_off.check_Basis(repair=True)
supmrs_off.rescaleForFitting()

res_sup_off = supmrs_off.fit(
    ppmlim=(0.2, 4.2),
    baseline='spline, moderate'
)

plotly_fit(supmrs_off, res_sup_off, (0.2, 4.2))

In [ ]:
#Fit the water resonance in the unsuppressed data

basis_off = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/uzay_svs_mpress_68_off_h2o')
mrs1 = off_unsup.mrs(basis=basis_off)

mrs1.keep = ['H2O']

mrs1.check_Basis(repair=True)
mrs1.rescaleForFitting()


#Fitted over limited frequency range with no baseline
res_water = mrs1.fit(
    ppmlim = (-10, 4.55),
    baseline = 'off',
    model='voigt'
)



wat_concs = res_water.getConc()[0] / (mrs1.fid_scaling / (mrs1.basis_scaling[0]))

plt.figure()
_ = res_water.plot(mrs1)

In [ ]:
#Subtract the fitted water peak from the raw data 


from fsl_mrs.utils import misc


fid_pred = res_water.predictedFID(mrs1, mode='Full', noBaseline=True)
fid_pred  /= mrs1.scaling['FID']

water = off_unsup.copy()
water[:] = fid_pred

metabs = proc.subtract(off_unsup, water)
#proc.subtract has a division by 2, so need to mutiply back by 2
metabs[:] *=2

metabs = proc.shift_to_reference(metabs, 2.01, (1.9, 2.1), figure=False)

In [ ]:
#Creating a nifti_mrs of the right shape for alingment
x = off_sup.copy()
a = x.copy()
b = x.copy()

# 2) Create a singleton 5th dimension
a = core.nifti_mrs.reshape(a, (1,), d5='DIM_EDIT')
b = core.nifti_mrs.reshape(b, (1,), d5='DIM_EDIT')

# 3) Merge them along the 5th dimension → size 2
x = core.nifti_mrs.merge([a, b], dimension='DIM_EDIT')

x[:,:,:,:,0] = metabs[:]
x[:,:,:,:,1] = off_sup[:]



#Alignment of the unsuppressede to the water suppressed data
x = proc.align(x, 'DIM_EDIT', ppmlim=(0.2, 4.2))


y = metabs.copy()
y[:] = x[:,:,:,:,0]
z = metabs.copy()
z[:] = x[:,:,:,:,1]

#Optional saving for later plotting
#y.save(f'{NOTEBOOK_DIR}/Data_for_plotting/mPress_raw/{part}_unsup.nii.gz')
#z.save(f'{NOTEBOOK_DIR}/Data_for_plotting/mPress_raw/{part}_sup.nii.gz')
#x.save(f'{NOTEBOOK_DIR}/Data_for_plotting/mPress_raw/{part}_difference.nii.gz')

plt.figure(figsize=(6,4))
y.plot(ppmlim=(0,4.2))
z.plot(ppmlim=(0,4.2))
x.plot(ppmlim=(0,4.2))

ax = plt.gca()

# Recolor lines
lines = ax.get_lines()

diff_line = lines[2]
xdata = diff_line.get_xdata()
ydata = diff_line.get_ydata()

#Multiply by 2 due to proc.subtract reasons and then vertically offset
ydata_new = (2 * ydata - 1e-5)

# Update the line
diff_line.set_ydata(ydata_new)

lines[1].set_color('blue')
lines[0].set_color('red')
lines[2].set_color('orange')

lines[2].set_linewidth(1.8)
lines[1].set_linewidth(1.8)
lines[0].set_linewidth(1.8)

lines[1].set_linestyle((0,(1,1)))

ax.legend(
    [lines[0], lines[1], lines[2]],
    ['GIRF-Corrected Non-WS', 'WS', 'Difference'],
    fontsize=10
)

# Remove y ticks and set label
ax.set_yticks([])
ax.axhline(-1e-5, color='gray', linestyle=':', linewidth=0.7)

#ax.text(ax.get_xlim()[0] - 0.1, 0, '0', va='center', ha='right', fontsize=10, color='gray')
ax.set_ylim(-0.000018, 0.00005)
ax.set_ylabel('Signal', fontsize=11)
ax.set_xlabel("Chemical Shift (ppm)", fontsize=11)
ax.tick_params(axis='x', labelsize=8)
ax.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.6)
ax.set_title('Processed Data for Metabolite Fitting', fontsize = 14)
#ax.set_title('HLSVD Removal of Residual Water', fontsize = 14)
ax.grid(False)  # turns all grids off first
ax.xaxis.grid(False, which='major')
ax.xaxis.grid(False, which='minor')

plt.tight_layout()
plt.show()


In [ ]:
#Fit the water unsuppressed data

metabs[:] = x[:,:,:,:,0]


basis_nowater = mrs_io.read_basis(rf'{NOTEBOOK_DIR}/basis_sets/uzay_svs_mpress_68_off_h2o')
basis_nowater.remove_fid_from_basis('H2O')

unsupmrs_off = metabs.mrs(basis=basis_nowater)
unsupmrs_off.check_Basis(repair=True)
unsupmrs_off.rescaleForFitting()

unsupmrs_off.fid_scaling = supmrs_off.fid_scaling


res_unsup_off = unsupmrs_off.fit(
    ppmlim=(0.2, 4.2),
    baseline='spline, moderate',
    model='voigt'
)

plotly_fit(unsupmrs_off, res_unsup_off, (0.2, 4.2))

In [ ]:
x = off_sup.copy()
a = x.copy()
b = x.copy()

# 2) Create a singleton 5th dimension
a = core.nifti_mrs.reshape(a, (1,), d5='DIM_EDIT')
b = core.nifti_mrs.reshape(b, (1,), d5='DIM_EDIT')

# 3) Merge them along the 5th dimension → size 2
x = core.nifti_mrs.merge([a, b], dimension='DIM_EDIT')

#appropriate scale factor
sf2 = supmrs_off.fid_scaling / unsupmrs_off.fid_scaling

x[:,:,:,:,0] = res_unsup_off.predictedFID(unsupmrs_off, noBaseline=True)
x[:,:,:,:,1] = res_sup_off.predictedFID(supmrs_off, noBaseline=True) / sf2

x = proc.align(x, 'DIM_EDIT', ppmlim=(0.2, 4.2))


y = metabs.copy()
y[:] = x[:,:,:,:,0]
z = metabs.copy()
z[:] = x[:,:,:,:,1]

#Optional saving for later plotting
#y.save(f'{NOTEBOOK_DIR}/Data_for_plotting/mPress_fitted/{part}_unsup.nii.gz')
#z.save(f'{NOTEBOOK_DIR}/Data_for_plotting/mPress_fitted/{part}_sup.nii.gz')
#x.save(f'{NOTEBOOK_DIR}/Data_for_plotting/mPress_fitted/{part}_difference.nii.gz')

plt.figure(figsize=(6,4))
y.plot(ppmlim=(0,4.2))
z.plot(ppmlim=(0,4.2))
x.plot(ppmlim=(0,4.2))

ax = plt.gca()

# Recolor lines
lines = ax.get_lines()

diff_line = lines[2]
xdata = diff_line.get_xdata()
ydata = diff_line.get_ydata()

# Apply your modifications:
#   1) multiply by 2
#   2) offset downward by 1e-5
ydata_new = (2 * ydata - 5)

# Update the line
diff_line.set_ydata(ydata_new)

lines[1].set_color('blue')
lines[0].set_color('red')
lines[2].set_color('orange')

lines[2].set_linewidth(1.8)
lines[1].set_linewidth(1.8)
lines[0].set_linewidth(1.8)

lines[1].set_linestyle((0,(1,1)))

ax.legend(
    [lines[0], lines[1], lines[2]],
    ['GIRF-Corrected Non-WS', 'WS', 'Difference'],
    fontsize=10
)

# Remove y ticks and set label
ax.set_yticks([])
ax.axhline(-5, color='gray', linestyle=':', linewidth=0.7)

#ax.text(ax.get_xlim()[0] - 0.1, 0, '0', va='center', ha='right', fontsize=10, color='gray')
ax.set_ylim(-10, 40)
ax.set_ylabel('Signal', fontsize=11)
ax.set_xlabel("Chemical Shift (ppm)", fontsize=11)
ax.tick_params(axis='x', labelsize=8)
#ax.axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.6)
ax.set_title('Fitted Metabolites', fontsize = 14)
#ax.set_title('HLSVD Removal of Residual Water', fontsize = 14)
ax.grid(False)  # turns all grids off first
ax.xaxis.grid(False, which='major')
ax.xaxis.grid(False, which='minor')

plt.tight_layout()
plt.show()

In [ ]:
#Comparate metabolite ratios

import pandas as pd
pd.set_option('display.float_format', '{:.6f}'.format)

metab_conc_unsup = res_unsup_off.getConc() / (unsupmrs_off.fid_scaling / (unsupmrs_off.basis_scaling[0]))
metab_conc_sup = res_sup_off.getConc() / (supmrs_off.fid_scaling / (supmrs_off.basis_scaling[0]))


df = pd.DataFrame()
df.loc[:, 'metabs_unsup'] = pd.Series(metab_conc_unsup, res_unsup_off.metabs)
df.loc[:, 'metabs_sup'] = pd.Series(metab_conc_sup, res_sup_off.metabs)



# Add combined metabolite pools


df.loc['H2O', 'metabs_unsup'] = wat_concs
df.loc['H2O', 'metabs_sup'] = wat_concs

df.loc['tNAA'] = df.loc['NAA'] + df.loc['NAAG']
df.loc['tCho'] = df.loc['GPC'] + df.loc['PCh']
df.loc['Glx']  = df.loc['Glu'] + df.loc['Gln']

if 'Cr+PCr' in df.index:
    df.rename(index={'Cr+PCr': 'tCr'}, inplace=True)
df =df.div(df.loc['H2O',:])
df["unsup/sup"] = df["metabs_unsup"] / df["metabs_sup"]

print(df)

#Optional save
#df.to_csv(rf'{NOTEBOOK_DIR}/Data_for_plotting/concetration_ratios/mPress/{part}.csv', index=True)